# Выбор локации для скважины

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

План выполнения работы:
- В избранном регионе ищем месторождения, для каждого определяем значения признаков;
- Строим модель и оцениваем объём запасов;
- Выбираем месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**

Данные геологоразведки трёх регионов находятся в трех файлах:
- */datasets/geo_data_0.csv*
- */datasets/geo_data_1.csv*
- */datasets/geo_data_2.csv*

Столбцы в файлах:
- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
- `product` — объём запасов в скважине (тыс. баррелей).

**Условия задачи:**

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

Прочитаем файлы с данными о скважинах

In [2]:
try:
    df_region_1 = pd.read_csv('/datasets/geo_data_0.csv')
    df_region_2 = pd.read_csv('/datasets/geo_data_1.csv')
    df_region_3 = pd.read_csv('/datasets/geo_data_2.csv')
except FileNotFoundError:
    df_region_1 = pd.read_csv('geo_data_0.csv')
    df_region_2 = pd.read_csv('geo_data_1.csv')
    df_region_3 = pd.read_csv('geo_data_2.csv')    

Посмотрим на некоторые данные о наборах данных

In [3]:
df_region_1.info()
df_region_2.info()
df_region_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [4]:
df_region_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [5]:
df_region_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [6]:
df_region_3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [7]:
df_region_1.duplicated().sum()

0

In [8]:
df_region_2.duplicated().sum()

0

In [9]:
df_region_3.duplicated().sum()

0

Пропусков в данных нет, дубликатов тоже нет. Есть три признака f0, f1, f2 которые похоже уже прошли процедуру масштабирования. Так же есть ключевой признак - объём запасов в скважине. Столбец с id для обучния модели нам не понадобится.

In [10]:
FEATURES_COLUMNS = ['f0','f1','f2'] # имена колонок с признаками для обучения модели
TARGET_COLUMN = 'product' # ключевой признак

## Обучение и проверка модели

По условиям задачи будем использовать модель линейной регрессии 

In [11]:
# функция для обучения модели
def linear_model(df, ratio, features_col, target_col):   
    features = df[features_col] # берем признаки для обучения
    target = df[target_col] # ключевой признак
    # делим df на обучающую и валидационную выборки
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=ratio, random_state=314)
    model = LinearRegression() # создаем модель
    model.fit(features_train,target_train) # обучаем модель 
    predictions = model.predict(features_valid) # делаем предсказание 
    rmse = mean_squared_error(target_valid,predictions,squared=False) # считаем RMSE
    return model, rmse, predictions

In [12]:
model_reg_1, rmse_reg_1, predict_reg_1 = linear_model(df_region_1, 0.25, FEATURES_COLUMNS, TARGET_COLUMN)
model_reg_2, rmse_reg_2, predict_reg_2 = linear_model(df_region_2, 0.25, FEATURES_COLUMNS, TARGET_COLUMN)
model_reg_3, rmse_reg_3, predict_reg_3 = linear_model(df_region_3, 0.25, FEATURES_COLUMNS, TARGET_COLUMN)

In [13]:
print('Средний запас предсказанного сырья по скважинам в первом регионе', round(predict_reg_1.mean(),2),'и среднеквадратичная ошибка модели',round(rmse_reg_1,2))
print('Средний запас предсказанного сырья по скважинам во втором регионе', round(predict_reg_2.mean(),2),'и среднеквадратичная ошибка модели',round(rmse_reg_2,2))
print('Средний запас предсказанного сырья по скважинам в третьем регионе', round(predict_reg_3.mean(),2),'и среднеквадратичная ошибка модели',round(rmse_reg_3,2))

Средний запас предсказанного сырья по скважинам в первом регионе 92.78 и среднеквадратичная ошибка модели 37.73
Средний запас предсказанного сырья по скважинам во втором регионе 68.75 и среднеквадратичная ошибка модели 0.89
Средний запас предсказанного сырья по скважинам в третьем регионе 94.95 и среднеквадратичная ошибка модели 39.96


В первом и третьем регионах модели машинного обучения показывают примерно одинаковые результаты. Среднеквадратичная ошибка получилась около 40 единиц. Во втором регионе ошибка близка к значению в 1 единицу. Также стоит отметит, что в 1 и 3 регионах средней запас сырья предсказанноо моделью больше чем во втором регионе. 

Во втором регионе модель предсказывает точнее.

## Подготовка к расчёту прибыли

Для расчета прибыли нам понадобиться значение затрачиваемое на разработку скважин в регионе — 10 млрд рублей. И стоимость продукта. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей. 

Затраты на разработку скважин в регионе - 10 000 000 000 рублей.

Доход с единицы продукат - 450 000 рублей.

Так же для расчета прибыли необходимы значения количества скважит которые разведуют и количество которое выбирают с помощью прогноза по запасам с помощью модели машинного обучения.

Разведуют скважин - 500 штук.

Выбирают лучших по результатам прогноза запасов для разработки - 200 штук.

In [14]:
INVEST_REG = 10_000_000_000 # размер инвестиций в разработку скважин одного региона
PRICE_OIL = 450_000 # стоимость одной единицы продукции
NUM_HOLE = 500 # кол-во разведываемых скважин
NUM_BEST_HOLE = 200 # кол-во лучших из разведанных скважин выбранных для разработки

Напишем функцию которая будет по заданному набору запасов в скважинах расчитывать прибыль

In [15]:
# функция для расчета прибыли
def profit(df, num_best, price, invest):
    result = - invest  # учет инвестиций
    df = df.sort_values(by='predictions', ascending=False) # сортируем в порядке убывания
    for i in range(0,num_best): # для первых 200 скважин, с максимальными запасами, считаем прибыль
        result += df['product'].values[i] * price
    return result

Посчитаем необходимый средний запас сырья в каждой из 200 разрабатываемых скважин для получения прибыли.

Получим простую формулу 

Средний запас >  Инвестиции / (Кол-во скважин * стоимость ед. запаса) 

In [16]:
print('Минимальное среднее кол-во запасов в разрабатываемой скважине для получения прибыли',round(INVEST_REG / (NUM_BEST_HOLE * PRICE_OIL),2)) 

Минимальное среднее кол-во запасов в разрабатываемой скважине для получения прибыли 111.11


Учитывая наши предсказания наших моделей по средним запасам в трех регионах, получить прибыль будет не так-то просто. Второй регион выглядит наименее выигрышным, но зато в нем разброс точности предсказания существенно выше чем в 1 и 3 регионе.

## Расчёт прибыли и рисков 

Для расчета возможных прибыли и убытков воспользуемся методом бутсрапа. 
Будем делать 1000 выборок по 500 элементов в каждой, из 500 выбранных скважин обученной моделью получим предсказание запасов и выберем 200 скважин с наибольшими запасами, после чего посчитаем прибыль при разработке этих 200 скважин.

In [17]:
state = RandomState(314)
num_samples = 1000 # количество выборок для bootstrap

In [18]:
# функция для проведения процедуры bootstrap
def bootstrap(df, samples, num , model): 
    global NUM_BEST_HOLE 
    global PRICE_OIL
    global INVEST_REG
    global FEATURES_COLUMNS
    values = [] # пустой список для значений выручки в каждой из выборок
    for i in range(samples): # цикл для проведения выборок
        sub_df = df.sample(n=num, replace=True, random_state=state) # формируем выборку
        pred = model.predict(sub_df[FEATURES_COLUMNS]) # делаем для выборки предсказание запасов 
        sub_df['predictions'] = pred
        prof = profit(sub_df, NUM_BEST_HOLE,  PRICE_OIL, INVEST_REG) # считаем для лучших 200 скважин из выборки прибыль
        values.append(prof) # добавляет прибыль в список
    values = pd.Series(values)
    q_25 = values.quantile(0.025) # считаем квантиль 2.5%
    q_975 = values.quantile(0.975) # считаем квантиль 97.5%
    print('Средняя прибыль: {:.0f} млн. рублей'.format(values.mean()/1000))
    print('Максимальная прибыль: {:.0f} млн. рублей' .format(values.max()/1000))
    print('Максимальный убыток: {:.0f} млн. рублей'.format(values.min()/1000))
    print('Квантиль 2.5% : {:.0f} млн. рублей'.format(q_25/1000))
    print('Квантиль 97.5% : {:.0f} млн. рублей'.format(q_975/1000))
    print('95% интервал прибыли: {:.0f} - {:.0f} млн. рублей'.format(q_25/1000,q_975/1000))
    print('Вероятность убытков', (values<0).mean())

Регион № 1

In [19]:
bootstrap(df_region_1, num_samples, NUM_HOLE, model_reg_1)

Средняя прибыль: 429069 млн. рублей
Максимальная прибыль: 1336758 млн. рублей
Максимальный убыток: -395126 млн. рублей
Квантиль 2.5% : -108960 млн. рублей
Квантиль 97.5% : 907243 млн. рублей
95% интервал прибыли: -108960 - 907243 млн. рублей
Вероятность убытков 0.049


Регион № 2

In [20]:
bootstrap(df_region_2, num_samples, NUM_HOLE, model_reg_2)

Средняя прибыль: 461932 млн. рублей
Максимальная прибыль: 1043518 млн. рублей
Максимальный убыток: -135853 млн. рублей
Квантиль 2.5% : 35798 млн. рублей
Квантиль 97.5% : 860304 млн. рублей
95% интервал прибыли: 35798 - 860304 млн. рублей
Вероятность убытков 0.016


Регион № 3

In [21]:
bootstrap(df_region_3, num_samples, NUM_HOLE, model_reg_3)

Средняя прибыль: 372174 млн. рублей
Максимальная прибыль: 1288918 млн. рублей
Максимальный убыток: -397454 млн. рублей
Квантиль 2.5% : -137383 млн. рублей
Квантиль 97.5% : 878302 млн. рублей
95% интервал прибыли: -137383 - 878302 млн. рублей
Вероятность убытков 0.074



Регион № 1 с наибольшей максимальной прибылью, но со вторым значением средний прибыли.
Регион № 2 выглядит самым прибыльным, и с наименьшей вероятностью убытков
Регион № 3 самым рискованным.

Из трех регионов только второй соответствует критерию убыточности менее 2.5% 
Если смотреть на среднюю прибыль то стоит остановиться на втором регионе. Так же стоит учесть, что именно модель для второго региона показана наилучший результат для точности предсказания запасов, менее 1 единицы.

## Вывод

Обучили модели для каждого региона.

С помощью метода bootstrap посчитали среднюю выручку, 95 % доверительный интервал и вероятность убытков.
В итоге учитывая постановку задачи для разведки и разработки скважин лучшим выглядит второй регион.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [X]  Ячейки с кодом расположены в порядке исполнения
- [X]  Выполнен шаг 1: данные подготовлены
- [X]  Выполнен шаг 2: модели обучены и проверены
    - [X]  Данные корректно разбиты на обучающую и валидационную выборки
    - [X]  Модели обучены, предсказания сделаны
    - [X]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [X]  На экране напечатаны результаты
    - [X]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован